In [161]:
from manim import *
import math
import numpy as np
import random

In [218]:
%%manim -qm MoleculeFission
class Atom(VGroup):
    #Atom is a VGroup of circles
    def __init__(self, size=0.08,num=6,**kwargs):
        super().__init__(**kwargs)
        #array of circle that make up atom, num describes how many circle around the center. Recommended even numbers
        self.array = [Circle(radius=size/2,color=rgb_to_color([(256/3*((i%3)))/256,(256/3*((i+1)%3))/256,(256/3*((i+2)%3))/256]),fill_opacity=1).move_to((size*math.cos((360/num)/180*PI*i),size*math.sin((360/num)/180*PI*i),0)) for i in range(num)]
        self.size = size
        self.num = num
        self.center = (0,0,0) #get center of atom, useful for stuff later
        for i in range(len(self.array)):
            super().add(self.array[i])

class Molecule(VGroup):
    #Molecule is a VGroup of Atoms
    def __init__(self, size=3,**kwargs):
        super().__init__(**kwargs)
        if(size%2!=1):
            size+=1 #need odd size else there is no "center"
        self.xbuff = 14/size #get x offset from center
        self.ybuff = 8/size #get y offset from center
        self.topleft = (0-self.xbuff*math.floor(size/2),0+self.ybuff*math.floor(size/2),0) #get top left point as reference
        self.array = [Atom() for i in range(size*size)] #array of atoms
        self.size = size
        self.fqu = [] #fission queue: used later
        self.fquc = [0]*size**2 #array to keep track if the atom has gone through fission
        super().add(self.array[0].move_to(self.topleft))
        self.array[0].center=self.topleft
        for i in range(1,len(self.array)):
            pos = tuple(np.add(self.topleft,(self.xbuff*(i%size),-self.ybuff*math.floor(i/size),0))) #initialize in grid pattern with respect to top left
            super().add(self.array[i].move_to(pos))
            self.array[i].center = pos #set new center for each atom
    
def checknum(index,high,low): #for checking if index is defined
    if(index<high and index>low):
        return True
    return False
def checkadj(i1,i2,size): #for checking if the atom are adjacent to each other (otherwise cross screen fission could happen)
    return abs(i2%size-i1%size)<2
class MoleculeFission(Scene):
    def fission(self,atom,runtime=0.5): #fission function
        animation = []
        for i in range(atom.num):    
            if (i > math.floor(atom.num/2)-1):
                animation.append(atom.array[i].animate(run_time=runtime).shift((0,-atom.size,0)))
            else:
                animation.append(atom.array[i].animate(run_time=runtime).shift((0,atom.size,0)))
        tempcr = Circle(radius=atom.size*4,color=WHITE,fill_opacity=0.5).move_to(atom.center)
        animation.append(Create(tempcr))
        return animation,tempcr
    def crit(self,mole,ty): #criticality function, ty: 0 for subcritical, 1 for critical, 2 for supercritical
        arr = []
        arrt = []
        arrt2 = []
        mole.fqu.append(math.floor(mole.size**2/2))
        amt = []
        amt.append(1)
        while mole.fqu:
            suma = 0
            for i in range(amt.pop(0)):
                if(mole.fqu):
                    (temp,tempcr),temp2,amtt = self.critr(mole.fqu.pop(0),mole,ty)
                    suma+=amtt
                    if (not(temp is None)):
                        arr += temp
                        arrt += tempcr
                        if(not(temp2 is None)):
                            arrt2 += temp2
            amt.append(suma)
            if(arr):
                self.play(*arr)
                for i in range(len(arrt)):
                    self.remove(arrt[i])
                if(arrt2):
                    self.play(*arrt2)
                arr=[]
                arrt = []
                arrt2=[]

            
    def critr(self,index,mole,ty,amt=0): #criticality helper function
        choices = [-mole.size-1,-mole.size,-mole.size+1,-1,1,mole.size-1,mole.size,mole.size+1]
        if(checknum(index,len(mole.array),0) and mole.fquc[index]==0):
            mole.fquc[index]+=1
            if(ty==2):
                arr = []
                for i in range(5):
                    temp = index+random.choice(choices)
                    chk = []
                    if(checknum(temp,len(mole.array),0) and mole.fquc[temp]==0 and checkadj(index,temp,mole.size) and not(temp in chk)):
                        chk.append(temp)
                        mole.fqu.append(temp)
                        arr+=self.neutron(mole,index,temp)
                        amt+=1
                return self.fission(mole.array[index]),arr,amt
            elif(ty==1):
                temp = index+random.choice(choices)
                while (not (checknum(temp,len(mole.array),0)) or not checkadj(index,temp,mole.size)):
                    temp = index+random.choice(choice)
                mole.fqu.append(temp)
                if(mole.fquc[temp]==0):
                    return self.fission(mole.array[index]),self.neutron(mole,index,temp),amt
                else:
                    return self.fission(mole.array[index]),None,amt
            elif(ty==0):
                if(random.randint(0,3)>1):
                    temp = index+random.choice(choices)
                    while (not (checknum(temp,len(mole.array),0)) or not checkadj(index,temp,mole.size)):
                        temp = index+random.choice(choices)
                    mole.fqu.append(temp)
                    if(mole.fquc[temp]==0):
                        return self.fission(mole.array[index]),self.neutron(mole,index,temp),amt
                    else:
                        return self.fission(mole.array[index]),None,amt
                else:
                    return (None,'a'),'a',amt
            else:
                return (None,'a'),'a',amt
        else:
            return (None,'a'),'a',amt
    def neutron(self,mole,i1,i2): #draw the path of each neutron to show how fission proceeds
        animation = []
        if(checknum(i2,len(mole.array),0)):
            d1 = Dot().set_color(GRAY).move_to(mole.array[i1].center)
            animation.append(FadeOut(d1, target_position=mole.array[i2].center))
            return animation
    def construct(self):
        molecule = Molecule(size=5)
        self.add(molecule)
        self.crit(molecule,2)
            
        

[09/30/24 21:12:12] INFO     Animation 0 : Partial movie file written in                   ]8;id=280253;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=630659;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/edriklee/Downloads/media/videos/Downloads/720p30/part                         
                             ial_movie_files/MoleculeFission/274514146_2525294373_10808197                         
                             66.mp4'                                                                               

[09/30/24 21:12:14] INFO     Animation 1 : Partial movie file written in                   ]8;id=474341;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=545141;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/edriklee/Downloads/media/videos/Downloads/720p30/part                         
                             ial_movie_files/MoleculeFission/2789044632_3105167508_3996356                         
                             552.mp4'                                                                              

[09/30/24 21:12:15] INFO     Animation 2 : Partial movie file written in                   ]8;id=573240;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=201805;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/edriklee/Downloads/media/videos/Downloads/720p30/part                         
                             ial_movie_files/MoleculeFission/2789044632_1053796749_3900468                         
                             317.mp4'                                                                              

[09/30/24 21:12:16] INFO     Animation 3 : Partial movie file written in                   ]8;id=571424;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=998890;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/edriklee/Downloads/media/videos/Downloads/720p30/part                         
                             ial_movie_files/MoleculeFission/2789044632_739685782_31986419                         
                             89.mp4'                                                                               

[09/30/24 21:12:18] INFO     Animation 4 : Partial movie file written in                   ]8;id=504761;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=509575;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/edriklee/Downloads/media/videos/Downloads/720p30/part                         
                             ial_movie_files/MoleculeFission/2789044632_4166010197_4370170                         
                             90.mp4'                                                                               

...

[09/30/24 21:12:21] INFO     Animation 7 : Partial movie file written in                   ]8;id=828135;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=447172;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/edriklee/Downloads/media/videos/Downloads/720p30/part                         
                             ial_movie_files/MoleculeFission/2789044632_1231846780_2714232                         
                             758.mp4'                                                                              

[09/30/24 21:12:23] INFO     Animation 8 : Partial movie file written in                   ]8;id=360839;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=758748;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/edriklee/Downloads/media/videos/Downloads/720p30/part                         
                             ial_movie_files/MoleculeFission/2789044632_2895109576_3004690                         
                             763.mp4'                                                                              

                    INFO     Combining to Movie file.                                      ]8;id=698085;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=393591;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#617\617]8;;\

                    INFO                                                                   ]8;id=517862;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=368677;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene_file_writer.py#736\736]8;;\
                             File ready at                                                                         
                             '/Users/edriklee/Downloads/media/videos/Downloads/720p30/Mole                         
                             culeFission.mp4'                                                                      
                                                                                                                   

                    INFO     Rendered MoleculeFission                                                  ]8;id=301680;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=880828;file:////anaconda3/lib/python3.10/site-packages/manim/scene/scene.py#241\241]8;;\
                             Played 9 animations                                                                   